<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1112hagoromo_masked_sen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「はごろも」をもちいた T5 による文章穴埋め問題作成の試み

- datafile: `masked_sen.xlsx`   # to be uploaded 
- date: 2022_1112
- author: 浅川伸一
- filename: `2022_1112hagoromo_masked_sen.ipynb`

「はごろも」とは岩下先生からいただいた，日本語教育用の文法，用法などをまとめたデータベースであるらしい。
本コードは，[T5 による，文章穴埋め問題](https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0918T5_demo_filling_blank_question.ipynb) の続編として意図されたものである。

### 使用上の注意

* `[CLS]` を使用すること。`[CLS]` はモデルの学習時に使用されるため，マスクされたトークンを予測するには，必ず文の前に `[CLS]` トークンを追加して，モデルが正しく符号化できるようにする。
* トークン化した後に `[MASK]` を使用する。入力文字列に直接 [MASK] を入力するのと，トークン化した後に `[MASK]` で置き換えるのとでは、トークンの並びが異なるので，予測結果も異なる。
トークン化後に `[MASK]` を使用する方がよい。
これは，モデルがどのように事前学習されたかと一致するためである。
しかし Huggingface Inference API は入力文字列に `[MASK]` をタイプすることしかサポートしておらず，よりロバストな予測を生成することができない。
* `position_ids` を明示的に引数として与える。
`Roberta` モデルでは，モデルに対して `position_ids` が与えられない場合 Huggingface 版の transformers は自動的に `position_ids` を構築する。だが， 0 ではなく `padding_idx` から作成する。これでは `rinna/japanese-roberta-base` では期待通りに動作しない。
なぜなら対応するトークナイザーの `padding_idx` が 0 ではないからである。
そのため，必ず自分で `position_ids` を `constrcut` して，位置 ID 0 から開始するようにすること。

- 参照URL: [【日本語モデル付き】2021年に自然言語処理をする人にお勧めしたい事前学習済みモデル](https://qiita.com/sonoisa/items/a9af64ff641f0bbfed44)
- [同 colab](https://colab.research.google.com/github/sonoisa/t5-japanese/blob/main/t5_japanese_article_generation_inference.ipynb)
- [HuggingFace Hub](https://huggingface.co/sonoisa/t5-base-japanese)


In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install --upgrade 'fugashi[ipadic]' > /dev/null 2>&1
    !pip install --upgrade 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download
    !pip install --upgrade ipadic > /dev/null 2>&1
    !pip install --upgrade sentencepiece > /dev/null 2>&1
    !pip install --upgrade transformers > /dev/null 2>&1

    !pip install 'konoha[all]' > /dev/null 2>&1
    #!pip install --upgrade termcolor > /dev/null 2>&1
    !pip install --upgrade jaconv  > /dev/null 2>&1      
    !pip install ipynbname --upgrade > /dev/null 2>&1
    !git clone https://github.com/ShinAsakawa/bit.git
    import bit

isColab = bit.isColab
HOME = bit.HOME

In [ ]:
import torch
from transformers import T5Tokenizer 
from transformers import RobertaForMaskedLM
from transformers import T5ForConditionalGeneration

import os
import pandas as pd
import jaconv  
import sys

from termcolor import colored
try:
    import jaconv
except ImportError:
    !pip install jaconv
    import jaconv
    
import warnings

def load_excel(path: str):
    """Load data from an Excel file."""
    warnings.simplefilter(action='ignore', category=UserWarning)
    return pd.read_excel(path)

#MODEL_DIR = "sonoisa/t5-base-japanese-article-generation"
MODEL_DIR = "rinna/japanese-roberta-base"
# load tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

# load model
if MODEL_DIR == "sonoisa/t5-base-japanese-article-generation":
    model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)
elif MODEL_DIR ==  "rinna/japanese-roberta-base":
    model = RobertaForMaskedLM.from_pretrained(MODEL_DIR)

model = model.eval()

# 「はごろも」の読み込み

`masked_sen.xlsx` をアップロードする

In [ ]:
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `masekd_sen.xlsx` を指定してアップロード
    data_dir = '.'

In [ ]:
masked_hagoromo_fname = 'masked_sen.xlsx'
masked_hag = load_excel(masked_hagoromo_fname)
masked_hagoromo_dict = masked_hag.to_dict(orient='index')
masked_hag_dict = masked_hag[['例文','masked_sen']].to_dict(orient='index')

In [ ]:
verbose = True
special_tokens = ['<s>', '</s>',  '[SEP]', '[PAD]', '[CLS]', '[MASK]', '。', '、', '「','」'],
top_n = 5

for i, (k, v) in enumerate(masked_hagoromo_dict.items()):
    sent = v['例文']
    masked_sent = v['masked_sen'].replace('[mask]','[MASK]')
    masked_sent = '[CLS]' + masked_sent
    print(colored(f'{i:2d} 入力文       :{sent}', color='blue', attrs=['bold'])) if verbose else None
    print(f'   マスク化入力文:{masked_sent}') if verbose else None
    m_tokens = tokenizer.tokenize(masked_sent)

    # トークン id に変換
    m_token_ids = tokenizer.convert_tokens_to_ids(m_tokens)
    
    # テンソルに変換
    m_token_tensor = torch.LongTensor([m_token_ids])

    # position ids を作成
    m_position_ids = list(range(0, m_token_tensor.size(1)))

    m_position_id_tensor = torch.LongTensor([m_position_ids])
    
    # マスクに対応する上位 top_n 個の候補を取得
    m_outputs = model(input_ids=m_token_tensor, 
                    position_ids=m_position_id_tensor)
    m_preds = m_outputs[0]
    
    for idx in range(m_preds.size(1)):
        if m_tokens[idx] == tokenizer.mask_token:
            cands = m_preds[0][idx].topk(top_n)
            print(colored(tokenizer.convert_ids_to_tokens(cands.indices), 
                          color='red',
                          attrs=['bold']), 
                  end="")
        else:
            print(m_tokens[idx], end="")
            
    print()
    

In [ ]:
print(colored('ほげ', color='red'))